In [16]:
import os
import numpy as np
import cv2

name_li = ['brmh','kyuh','gugh']
fppi_cnt = 0
sum_images = 0
ai_li=[]
for name in name_li:
    gt_base = './data/img/result/{}_healthcheck/gt_mask'.format(name)
    base = './data/img/ai_json(all_files)/{}_healthcheck/ai_mask'.format(name)
    ai_mask_list = os.listdir(base)
    sum_image = len(ai_mask_list)
    sum_images+=sum_image

    
    for index, ai_mask in enumerate(sorted(ai_mask_list)):
        # if index==10:
        #     break
        ai_li.append(ai_mask)
        gt_file = '{}_mask.png'.format(ai_mask.split('_cutoff')[0])
        gt_im = cv2.imread(os.path.join(gt_base, gt_file), cv2.IMREAD_ANYDEPTH | cv2.IMREAD_GRAYSCALE)
        gt_ims.append(gt_im)
        w,h=gt_im.shape
    #     print(h,w)
    #     print("started reading gt file: {}".format(gt_file))

        im = cv2.imread(os.path.join(base, ai_mask))
        im = cv2.resize(im, (h,w),interpolation=cv2.INTER_NEAREST)
        ai_ims.append(im)
        imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
        _, ai_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #     print(gt_im.shape, im.shape)
        if gt_im.sum() == 0: # normal
            fppi_cnt += len(ai_contours)

        else:
            for ai_contour in ai_contours:
                _image = np.zeros_like(gt_im)
                _image = cv2.drawContours(_image, [ai_contour], 0, color=255, thickness=-1)
                if _image[gt_im>0].sum() == 0:
                    fppi_cnt += 1
    #     print("intermediate sum ffpi cnt: {}".format(fppi_cnt))


    if name == 'gugh':
        fppi = fppi_cnt/sum_images
        print('Calculated FPPI: {}'.format(fppi))

        n_bootstraps = 10
        rng_seed = 42  # control reproducibility
        bootstrapped_scores = []

        rng = np.random.RandomState(rng_seed)
        for m in range(n_bootstraps):
            fppi_cnt = 0

        #     y_true = np.ndarray(gt_ims)
        #     y_pred = np.ndarray(ai_ims)
            ai_li = np.array(ai_li)

            # bootstrap by sampling with replacement on the prediction indices
            indices = rng.randint(0, len(y_pred), len(y_pred))

            if len(np.unique(ai_li[indices])) < 2:
                # We need at least one positive and one negative sample for ROC AUC
                # to be defined: reject the sample
                continue
                
            for name_ci in ai_li[indices]:
                
                if name_ci.split('-')[0] == 'B':
                    hospital_name = 'brmh'
                elif name_ci.split('-')[0] == 'K':
                    hospital_name = 'kyuh'
                elif name_ci.split('-')[0] == 'G':
                    hospital_name = 'gugh'
                
                gt_base = './data/img/result/{}_healthcheck/gt_mask'.format(hospital_name)
                base = './data/img/ai_json(all_files)/{}_healthcheck/ai_mask'.format(hospital_name)
                
                gt_file = '{}_mask.png'.format(name_ci.split('_cutoff')[0])
                gt_im = cv2.imread(os.path.join(gt_base, gt_file), cv2.IMREAD_ANYDEPTH | cv2.IMREAD_GRAYSCALE)
                w,h=gt_im.shape

                im = cv2.imread(os.path.join(base, name_ci))
                im = cv2.resize(im, (h,w),interpolation=cv2.INTER_NEAREST)
                imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
                ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
                _,ai_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                if gt_im.sum() == 0: # normal
                    fppi_cnt += len(ai_contours)

                else:
                    for ai_contour in ai_contours:
                        _image = np.zeros_like(gt_im)
                        _image = cv2.drawContours(_image, [ai_contour], 0, color=255, thickness=-1)
                        if _image[gt_im>0].sum() == 0:
                            fppi_cnt += 1

                score = fppi_cnt/1
                bootstrapped_scores.append(score)
        #     print("Bootstrap #{} ROC area: {:0.3f}".format(m + 1, score))

        sorted_scores = np.array(bootstrapped_scores)
        sorted_scores.sort()

        # Computing the lower and upper bound of the 90% confidence interval
        # You can change the bounds percentiles to 0.025 and 0.975 to get
        # a 95% confidence interval instead.
        confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
        confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
        print("FPPI = {:0.4f} [{:0.3f} - {:0.3}]".format(fppi, confidence_lower/sum_images, confidence_upper/sum_images))

Calculated FPPI: 0.3837920489296636
gugh
FPPI = 0.3838 [0.010 - 0.376]


In [25]:
fppi_score = np.array(fppi_score)
mean = fppi_score.mean()
std = fppi_score.std()
print(mean, std)
ci = 1.96*(std/math.sqrt(sum_image))

confidence_lower = mean-ci
confidence_upper = mean+ci
print("FPPI = {:0.4f} [{:0.3f} - {:0.3}]".format(fppi, confidence_lower, confidence_upper))

0.21450093418185076 0.12070606381169723
FPPI = 0.4188 [0.210 - 0.219]
